In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pickle
import random

/Users/freestylebk/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def unpickle(file):
    with open(file, 'rb') as fo:
        opened = pickle.load(fo, encoding='latin1')
    return opened

In [3]:
data, labels = [], []

for i in range(1,6):
    name = 'cifar-10-batches-py/data_batch_{}'.format(i)
    batch_data = unpickle(name)
    
    if len(data) > 0:
        data = np.vstack((data, batch_data['data']))
        labels = np.hstack((labels, batch_data['labels']))
    else :
        data = batch_data['data']
        labels = batch_data['labels']
    print(np.shape(data), np.shape(labels))

(10000, 3072) (10000,)
(20000, 3072) (20000,)
(30000, 3072) (30000,)
(40000, 3072) (40000,)
(50000, 3072) (50000,)


In [4]:
data_test = unpickle('cifar-10-batches-py/test_batch')['data']
labels_test = unpickle('cifar-10-batches-py/test_batch')['labels']

In [5]:
print(np.shape(data_test), np.shape(labels_test))

(10000, 3072) (10000,)


In [6]:
from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()
labels_1hot = np.copy(labels)
labels_1hot = encoder.fit_transform(labels_1hot)

In [7]:
np.shape(labels_1hot)

(50000, 10)

In [8]:
labels_test_1hot = np.copy(labels_test)
labels_test_1hot = encoder.fit_transform(labels_test_1hot)

In [9]:
np.shape(labels_test_1hot)

(10000, 10)

In [10]:
label_names = unpickle('cifar-10-batches-py/batches.meta')['label_names']

In [11]:
label_names

['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

In [12]:
labels[:5]

array([6, 9, 9, 4, 1])

In [13]:
X = tf.placeholder(tf.float32, [None, 3072])
X_img = tf.reshape(X, [-1, 32, 32, 3])
Y = tf.placeholder(tf.float32, [None, len(label_names)])

In [14]:
# 첫번째 레이어
W1 = tf.Variable(tf.random_normal([3, 3, 3, 32], stddev=0.01))
L1 = tf.nn.conv2d(X_img, W1, strides=[1, 1, 1, 1], padding='SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
# 16 by 16

In [15]:
# 두번째 레이어
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
# 8 by 8
L2_flat = tf.reshape(L2, [-1, 8 * 8 * 64])

In [16]:
# Fully connected layer
W3 = tf.get_variable("W3", shape=[8*8*64,len(label_names)], initializer=tf.contrib.layers.xavier_initializer())
b = tf.Variable(tf.random_normal([len(label_names)]))
logits = tf.matmul(L2_flat, W3) + b

In [31]:
learning_rate = 0.05
training_epochs = 15
batch_size = 1000

In [18]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [24]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

print('Learning started. It takes sometime.')

for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(len(data) / batch_size)

    for i in range(total_batch):
        batch_xs, batch_ys = data[i*batch_size : (i+1)*batch_size], labels_1hot[i*batch_size : (i+1)*batch_size]
        c, _ = sess.run([cost, optimizer], feed_dict={X: batch_xs, Y:batch_ys})
        avg_cost += c / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(c))

print('Learning Finished!')

# Test model and check accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy:', sess.run(accuracy, feed_dict={X: data_test, Y: labels_test_1hot}))

# # Get one and predict
# r = random.randint(0, mnist.test.num_examples - 1)
# print("Label: ", sess.run(tf.argmax(mnist.test.labels[r:r + 1], 1)))
# print("Prediction: ", sess.run(tf.argmax(logits, 1), feed_dict={X: mnist.test.images[r:r + 1]}))

Learning started. It takes sometime.
Epoch: 0001 cost = 1.761715770
Epoch: 0002 cost = 1.547606707
Epoch: 0003 cost = 1.409927487
Epoch: 0004 cost = 1.333868146
Epoch: 0005 cost = 1.275294900
Epoch: 0006 cost = 1.234145284
Epoch: 0007 cost = 1.203053474
Epoch: 0008 cost = 1.185065389
Epoch: 0009 cost = 1.129818559
Epoch: 0010 cost = 1.094058871
Epoch: 0011 cost = 1.071828723
Epoch: 0012 cost = 1.048571348
Epoch: 0013 cost = 1.030445576
Epoch: 0014 cost = 1.013914585
Epoch: 0015 cost = 1.005368233
Learning Finished!
Accuracy: 0.5829


In [25]:
keep_prob = tf.placeholder(tf.float32)

In [26]:
L1 = tf.nn.dropout(L1, keep_prob=keep_prob)
L2 = tf.nn.dropout(L2, keep_prob=keep_prob)

In [27]:
W3 = tf.Variable(tf.random_normal([3, 3, 64, 128], stddev=0.01))
L3 = tf.nn.conv2d(L2, W3, strides=[1, 1, 1, 1], padding='SAME')
L3 = tf.nn.relu(L3)
L3 = tf.nn.max_pool(L3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
L3 = tf.nn.dropout(L3, keep_prob=keep_prob)
L3 = tf.reshape(L3, [-1, 128 * 4 * 4])

In [28]:
# Fully connected layer
W4 = tf.get_variable("W4", shape=[128 * 4 * 4, 625],initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([625]))
L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)
L4 = tf.nn.dropout(L4, keep_prob=keep_prob)

In [29]:
W5 = tf.get_variable("W5", shape=[625, 10], initializer=tf.contrib.layers.xavier_initializer())
b5 = tf.Variable(tf.random_normal([10]))
logits = tf.matmul(L4, W5) + b5

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

print('Learning started. It takes sometime.')

for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(len(data) / batch_size)

    for i in range(total_batch):
        batch_xs, batch_ys = data[i*batch_size : (i+1)*batch_size], labels_1hot[i*batch_size : (i+1)*batch_size]
        c, _ = sess.run([cost, optimizer], feed_dict={X: batch_xs, Y:batch_ys, keep_prob:0.7})
        avg_cost += c / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(c))

print('Learning Finished!')

# Test model and check accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy:', sess.run(accuracy, feed_dict={X: data_test, Y: labels_test_1hot, keep_prob:1}))

# # Get one and predict
# r = random.randint(0, mnist.test.num_examples - 1)
# print("Label: ", sess.run(tf.argmax(mnist.test.labels[r:r + 1], 1)))
# print("Prediction: ", sess.run(tf.argmax(logits, 1), feed_dict={X: mnist.test.images[r:r + 1]}))

Learning started. It takes sometime.
Epoch: 0001 cost = 1.998749971
Epoch: 0002 cost = 1.583518028
Epoch: 0003 cost = 1.496066213
Epoch: 0004 cost = 1.377483487
